# VacationPy

In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
output_data_file = "weather.csv"
types_df = pd.read_csv(output_data_file)
types_df.head()

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Severo-Kuril'sk,RU,0,1587217941,89,50.68,156.12,32.32,8.25
1,Bengkulu,ID,100,1587217941,79,-3.80,102.27,79.66,1.19
2,Ullapool,GB,0,1587217941,56,57.90,-5.16,53.01,11.99
3,Saskylakh,RU,0,1587217941,84,71.92,114.08,8.35,5.59
4,Hermanus,ZA,0,1587217743,40,-34.42,19.23,86.00,8.01


In [4]:
gmaps.configure(api_key=g_key)
locations = types_df[["Latitude", "Longitude"]]
humidity = types_df["Humidity"].astype(float)
max_intensity = types_df["Humidity"].max()

In [5]:
fig = gmaps.figure(center=(25,0), zoom_level=2.25)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_intensity,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
ideal_df = types_df.loc[(types_df["Max Temperature"] <80) & (types_df["Max Temperature"] >50) & (types_df["Wind Speed"] <5) &
                        (types_df["Cloudiness"] <20)]
ideal_df.dropna
ideal_df.count()

City               13
Country            12
Cloudiness         13
Date               13
Humidity           13
Latitude           13
Longitude          13
Max Temperature    13
Wind Speed         13
dtype: int64

In [25]:
hotel_df = ideal_df.copy()
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,Chui,UY,11,1587217776,67,-33.70,-53.46,70.43,4.68,
1,Port Alfred,ZA,0,1587217949,88,-33.59,26.89,73.00,1.99,
2,Avarua,CK,5,1587217949,83,-21.21,-159.78,75.20,2.24,
3,Lüderitz,NaN,0,1587217966,53,-26.65,15.16,75.20,4.70,
4,La Linea,AR,0,1587217970,63,-27.62,-55.04,72.00,3.00,
5,Chajul,GT,0,1587217972,28,15.49,-91.03,70.41,1.90,
6,Puerto Escondido,MX,5,1587217848,88,15.85,-97.07,78.80,3.36,
7,Kruisfontein,ZA,0,1587217539,33,-34.00,24.73,78.12,4.63,
8,Valparaíso,CL,15,1587218003,77,-33.04,-71.63,59.00,3.36,
9,Atasū,KZ,10,1587217553,34,48.68,71.64,55.67,4.21,


In [26]:
latitude = hotel_df["Latitude"]
longitude = hotel_df["Longitude"]
length_i = len(hotel_df)

In [27]:
for i in range(length_i):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude[i]},{longitude[i]}",
        "radius": 5000,
        "types": "lodging",
        "keyword": "Hotel",
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()
    
    try: 
        results = response["results"][i]["name"]
        print(f"Nearest hotel: {results}")
        hotel_df.loc[hotel_df["City"] == hotel_df["City"][i], "Hotel Name"] = results
    except IndexError:
        print("No hotels available")

Nearest hotel: Bertelli Chuí Hotel
Nearest hotel: The Beach House
Nearest hotel: The Rarotongan Beach Resort & Lagoonarium
Nearest hotel: The Cormorant House
No hotels available
No hotels available
Nearest hotel: HOTEL ROCKAWAY
No hotels available
Nearest hotel: Via Via Hotel Restaurant belga
No hotels available
Nearest hotel: Hotel Acapulco Malibú
No hotels available
Nearest hotel: Mr.KINJO in MEIO


In [28]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,Chui,UY,11,1587217776,67,-33.70,-53.46,70.43,4.68,Bertelli Chuí Hotel
1,Port Alfred,ZA,0,1587217949,88,-33.59,26.89,73.00,1.99,The Beach House
2,Avarua,CK,5,1587217949,83,-21.21,-159.78,75.20,2.24,The Rarotongan Beach Resort & Lagoonarium
3,Lüderitz,NaN,0,1587217966,53,-26.65,15.16,75.20,4.70,The Cormorant House
4,Puerto Escondido,MX,5,1587217848,88,15.85,-97.07,78.80,3.36,HOTEL ROCKAWAY
5,Valparaíso,CL,15,1587218003,77,-33.04,-71.63,59.00,3.36,Via Via Hotel Restaurant belga
6,Acapulco de Juárez,MX,5,1587218011,64,16.86,-99.89,75.20,2.24,Hotel Acapulco Malibú
7,Nago,JP,1,1587218015,78,26.59,127.98,73.40,3.36,Mr.KINJO in MEIO


In [29]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [30]:
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))